<a href="https://colab.research.google.com/github/guvoltolino/computer-vision/blob/main/computer-vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ikomia #api ikomia para utilizar o yolo com deepSORT

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils.displayIO import display
from google.colab.patches import cv2_imshow
import cv2

video_name = 'pessoas_1'
video_path_input = f'videos/{video_name}.mp4'
video_path_output = f'videos-processados/{video_name}-processado.avi'


# Inicialize o fluxo de trabalho
workflow = Workflow()


# Adicione o algoritmo de detecção de objetos
object_detection_algorithm = workflow.add_task(name="infer_yolo_v7", auto_connect=True)


# Adicione o algoritmo de rastreamento ByteTrack
tracking_algorithm = workflow.add_task(name="infer_deepsort", auto_connect=True)


# Defina os parâmetros para o algoritmo de rastreamento
tracking_algorithm.set_parameters({
    "categories": "person",            # Categoria de objeto a ser rastreado
    "tracker_type": "bytetrack",       # Tipo de rastreador
    "conf_thres": "0.5",               # Limite de confiança para detecção
})


# Abra o arquivo de vídeo
video_capture = cv2.VideoCapture(video_path_input)
if not video_capture.isOpened():
    print("Erro: Não foi possível abrir o vídeo.")
    exit()


# Obtenha as propriedades do vídeo para a saída
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))   # Largura do quadro
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)) # Altura do quadro
fps = video_capture.get(cv2.CAP_PROP_FPS)                        # Taxa de quadros por segundo (FPS)


# Defina o codec e crie o objeto VideoWriter para salvar o vídeo processado
codec = cv2.VideoWriter_fourcc(*'XVID')                         # Codec de vídeo
video_writer = cv2.VideoWriter(video_path_output, codec, fps, (frame_width, frame_height))  # Objeto para salvar o vídeo


while True:
    # Leia um quadro do vídeo
    success, frame = video_capture.read()


    # Verifique se o vídeo terminou ou houve um erro
    if not success:
        print("Info: Fim do vídeo ou erro.")
        break


    # Execute o fluxo de trabalho no quadro atual
    workflow.run_on(array=frame)


    # Obtenha os resultados dos algoritmos
    processed_image = tracking_algorithm.get_output(0)  # Imagem processada com rastreamento
    detected_objects = tracking_algorithm.get_output(1) # Objetos detectados


    # Converta a imagem resultante para o espaço de cor BGR para exibição
    image_with_graphics = processed_image.get_image_with_graphics(detected_objects)
    display_image = cv2.cvtColor(image_with_graphics, cv2.COLOR_RGB2BGR)


    # Salve o quadro resultante no arquivo de saída
    video_writer.write(image_with_graphics)

    # Exiba o quadro processado
    # cv2_imshow(display_image)


    # Pressione 'q' para encerrar o processamento do vídeo
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Após o loop, libere todos os recursos
video_capture.release()
video_writer.release()
cv2.destroyAllWindows()

Will run on cpu
Fusing layers... 
Successfully loaded imagenet pretrained weights from "/root/Ikomia/Plugins/Python/infer_deepsort/models/checkpoints/osnet_x1_0_imagenet.pth"
Workflow Untitled run successfully in 2753.037472 ms.
Workflow Untitled run successfully in 2645.760828 ms.
Workflow Untitled run successfully in 1812.8491279999998 ms.
Workflow Untitled run successfully in 1778.718448 ms.
Workflow Untitled run successfully in 1843.853879 ms.
Workflow Untitled run successfully in 1800.138579 ms.
Workflow Untitled run successfully in 1806.598573 ms.
Workflow Untitled run successfully in 2438.654094 ms.
Workflow Untitled run successfully in 2993.1060669999997 ms.
Workflow Untitled run successfully in 2123.872374 ms.
Workflow Untitled run successfully in 1824.626108 ms.
Workflow Untitled run successfully in 1942.9722419999998 ms.
Workflow Untitled run successfully in 1904.226298 ms.
Workflow Untitled run successfully in 1921.783517 ms.
Workflow Untitled run successfully in 2393.12780